In [6]:
from math import ceil, sqrt
import random
from sympy.ntheory import factorint, discrete_log
from sympy.ntheory.modular import solve_congruence

# Pasos de bebe, pasos de gigante

In [7]:
def paso_bebe_paso_gigante_v1(g, y, n):
    m = ceil(pow(n, 1/2))
    g %= n + 1
    y %= n + 1

    tabla_hash = [(pow(g, j, n + 1), j) for j in range(m)]
    beta = pow(pow(g, -1, n + 1), m, n + 1)
    gamma = y
    
    for i in range(m):
        if any(a == gamma for a, _ in tabla_hash):
            x = i * m + next(b for a, b in tabla_hash if a == gamma)
            print(f'm = {m}\nix = {i}\njx = {next(b for a, b in tabla_hash if a == gamma)}\nx = {x}')
            break
        gamma = gamma * beta % (n + 1)
    else:
        x = None

    return x

In [8]:
def paso_bebe_paso_gigante_v2(
            g: int,
            y: int,
            n: int,
            debug_log = False
        ) -> int:

        m = ceil(sqrt(n))
        if debug_log: print("m = ", m)

        # lookup es una tabla hash para almacenar tuplas (a, b) \in G X Z
        # a es usada como clave indice
        lookup = {}

        # Computar (g**j, j) para todo 0 <= j <= m
        for j in range(m):
            lookup[pow(g, j, n)] =  j

        b = pow(g, m * (n - 2), n)

        _y = y
        x = None

        # Computar y * (g*{-m})i para i = 0, 1, 2, ... hasta entonces i_x con y * (g{-m})*i_x
        for i in range(m):
            if _y in lookup:
                x = (i * m + lookup[_y]) % n
                if debug_log: print(f"ix = {i} | jx = {lookup[_y]}")
                return x

            _y = (_y * b) % n
        return x

In [12]:
p = 3129553951
g = 19
y = 1132858435
n = p - 1

paso_bebe_paso_gigante_v1(g, y, n)

m = 55943
ix = 5725
jx = 1327
x = 320275002


320275002

In [13]:
p = 3129553951
g = 19
y = 1132858435
n = p - 1

paso_bebe_paso_gigante_v2(g, y, p)

320275002

# Pollard's Rho

In [ ]:
def MCD(x, y):
    if (x < y):
        b = x
        a = y
    else:
        a = x
        b = y
    while (b > 0):
        r = a % b
        a = b
        b = r
    return a

In [ ]:
def obtener_valor_binario(alpha):
    # Convierte alpha a su representación binaria como cadena de caracteres
    binario = bin(alpha)
    # Extrae el valor binario eliminando el prefijo '0b' y lo convierte a entero
    return int(binario[2:])

def fs(alpha, m):
    b_alpha = obtener_valor_binario(alpha)
    return b_alpha % m

In [ ]:
def almacenar_tabla(g, y, n, m):
    R = []
    g %= n + 1
    y %= n + 1

    for _ in range(1, m):
        uj = random.randint(0, n-1)
        vj = random.randint(0, n-1)
        aj = (g**uj * y**vj) % n
        R.append((aj, uj, vj))

    return R

In [ ]:
def caminata(xab, R, m, n):
    x, a, b = xab
    j = fs(x, m)
    if (j == 0):
        return (pow(x, 2, n), (2 * a) % n, (2 * b) % n)
    else:
        aj, uj, vj = R[j-1]
        return (aj * x % n, (a + uj) % n, (b + vj) % n)

In [ ]:
def pollard(g, y, n, m):
    R = almacenar_tabla(g, y, n, m)
    a0 = random.randint(0, n-1); b0 = random.randint(0, n-1); x0 = pow(g, a0) * pow(y, b0)

    (xi, ai, bi) = caminata((x0, a0, b0), R, m, n)
    (x2i, a2i, b2i) = caminata(caminata((x0, a0, b0), R, m, n), R, m, n)

    while (xi != x2i) :
        (xi, ai, bi) = caminata((xi, ai, bi), R, m, n)
        (x2i, a2i, b2i) = caminata(caminata((x2i, a2i, b2i), R, m, n), R, m, n)

    if (MCD(bi - b2i, n) == 1):
        return pow((a2i - ai) * (bi - b2i), -1, n)
    else:
        return 'Fallo'

In [ ]:
g = 2
y = 3
n = 17
m = 5

pollard(g, y, n, m)

'Fallo'

# Pohlig Hellman

In [ ]:
def pohlig_hellman(g, y, p):
    """
    Pohlig-Hellman Algorithm to solve discrete logarithms when the group order is a smooth number.
    
    Parameters:
    g : int
        Generator of the group.
    y : int
        Element of the group for which we are solving the discrete logarithm.
    p : int
        Order of the group.
        
    Returns:
    x : int
        The discrete logarithm of y base g.
    """
    
    # Factorize p-1 into prime factors
    factors = factorint(p - 1)
    # List to hold solutions to congruences
    congruences = []
    
    for q, e in factors.items():
        # q^e is the factor of p-1
        # Solve the discrete logarithm in the subgroup of order q^e
        gq = pow(g, (p - 1) // q**e, p)
        yq = pow(y, (p - 1) // q**e, p)
        # Compute the discrete logarithm modulo q^e
        dq = discrete_log(p, yq, gq, q**e)
        # Store the solution to the congruence
        congruences.append((dq, q**e))
        
    # Use the Chinese Remainder Theorem to solve the system of congruences
    x, _ = solve_congruence(*congruences)
    return x

In [ ]:
# Example to verify the implementation
# These values should be adjusted based on the specific problem
p = 3129553951
g = 19
y_example = 212313123 # Element to find log for

# Applying Pohlig-Hellman
discrete_log_result = pohlig_hellman(g, y_example, p)
discrete_log_result

1134497573